<a href="https://colab.research.google.com/github/GabHoo/GoTAutomyth_short/blob/main/Generate_with_BART_experiment_name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
experiment_name = 'events12_community_event'
#[event1_relation_all, events12_community_types, events12_community_range, events12_community_all]
#for event1_relation_all need old version

In [ ]:
### start with installing libraries
!pip install transformers datasets 
!pip install rouge.score nltk py7zr
!pip install evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.5 MB/s 
     |████████████████████████████████| 451 kB 62.3 MB/s 
     |████████████████████████████████| 182 kB 70.9 MB/s 
     |████████████████████████████████| 7.6 MB 48.2 MB/s 
     |████████████████████████████████| 212 kB 61.0 MB/s 
     |████████████████████████████████| 115 kB 63.9 MB/s 
     |████████████████████████████████| 127 kB 73.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |████████████████████████████████| 379 kB 14.3 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 93 kB 2.2 MB/s 
     |██████

In [ ]:
### import libraries

import transformers
from datasets import load_dataset, load_metric
from google.colab import drive
import numpy as np
import os
import nltk
import torch
import evaluate
nltk.download('punkt')
print(transformers.__version__)
print(torch.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


4.24.0
1.12.1+cu113


In [ ]:
### connect to the drive

MOUNTPOINT = '/content/gdrive'
Working_Dir = os.path.join(MOUNTPOINT, 'Shareddrives', 'GOT/Building_Narratives_Game_of_Thrones')
drive.mount(MOUNTPOINT)
print(Working_Dir)
#%cd /content/gdrive/Shareddrives/GOT/Building_Narratives_Game_of_Thrones/data


Mounted at /content/gdrive
/content/gdrive/Shareddrives/GOT/Building_Narratives_Game_of_Thrones


In [ ]:
### upload the data


train_file = Working_Dir + '/data/' + experiment_name + '/train_' + experiment_name +'.json'
dev_file = Working_Dir + '/data/' +  experiment_name + '/val_' + experiment_name + '.json'
test_file = Working_Dir + '/data/' + experiment_name + '/test_' + experiment_name + '.json'

dataset = load_dataset('json', data_files={'train': train_file, 'valid': dev_file, 'test': test_file})

### let's check the data

print(dataset['valid'][0]['story'])
print(dataset['valid'][0]['Knowledge Graph'])

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d6a2d63bc71c5056/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Saltpans was the place where Petyr Baelish, the Lord of House Arryn used to live. Petyr Baelish was also known around Saltpans as a great rebel . It was known that Margaery Tyrell wanted the  peace  that belonged to Petyr Baelish. It was sunrise in Great Sept of Baelor  when Margaery Tyrell  threatened the peace of Petyr Baelish . 
Lord - type - Male | Event_01 - hasOccupation - rebel | Event_02 - hasTime - sunrise | Event_02 - villain - Margaery_Tyrell | Event_02 - hasPlace - Great_Sept_of_Baelor | Event - type - Class | Event_01 - journeyStage - 1 | Lord - type - Title | Petyr_Baelish - type - Main_Character | sunrise - type - Time | Event_01 - hasActor - Petyr_Baelish | Event_01 - type - Event | Event_01 - hasPlace - Saltpans | Event_02 - ThreatenedElement - peace | Event_01 - type - OrdinaryWorld | Petyr_Baelish - type - Actor | peace - type - ThreatTarget | Event_02 - type - CallToAdventure | Event_02 - hasActor - Petyr_Baelish | Margaery_Tyrell - type - Main_Character | easter - 

In [ ]:
### data looks OK, let's continue with the tokenizer
max_input = 512
max_target = 512
model_checkpoints = 'facebook/bart-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [graph for graph in data_to_process['Knowledge Graph']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  #with tokenizer.as_target_tokenizer():
  targets = [target for target in data_to_process['story']]
  model_targets = tokenizer(targets, max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = model_targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [ ]:
tokenize_data = dataset.map(preprocess_data, batched = True) 

print(tokenize_data['test']['story'][3])
print(tokenize_data['test']['Knowledge Graph'][3])

### let's drop the columns that we won't need
tokenize_data = dataset.map(preprocess_data, batched = True, remove_columns=['story', 'Knowledge Graph'])
print(tokenize_data)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

 In The Iron Island Tyrion Lannister was a  King from the House Baratheon and worked as a advisor . Tyrion Lannister really cared about the treasure . It was the Summer Solstice in Torrhen's Square  when  Daenerys Targaryen threatened the  treasure of Tyrion Lannister . 
Event_01 - journeyStage - 1 | Daenerys_Targaryen - type - Actor | Event_01 - type - Event | the_Summer_Solstice - type - Time | Event_02 - ThreatenedElement - treasure | Daenerys_Targaryen - type - Female | Event_01 - hasActor - Tyrion_Lannister | Event_02 - type - Event | Event_02 - villain - Daenerys_Targaryen | advisor - type - Occupation | The_Iron_Island - type - Place | fictional_king - type - Actor | Tyrion_Lannister - type - Male | treasure - type - ThreatTarget | Event_02 - hasActor - Tyrion_Lannister | Event_01 - hasTitle - fictional_king | House_Baratheon - type - House | fictional_king - type - Title | Core - type - Class | Event_01 - hasHouse - House_Baratheon | Event_01 - hasPlace - The_Iron_Island | Even

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
})


In [ ]:
### load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
batch_size = 4
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
#####################
# metrics
# compute rouge for evaluation 
#####################
metric = load_metric('rouge')

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """


In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'output',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    generation_max_length = 128,
    generation_num_beams=5,
    eval_accumulation_steps=1,
    fp16=True
    )
#only CUDA available -> fp16=True

In [ ]:
### almost training time
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['valid'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

Using cuda_amp half precision backend


In [ ]:
### check GPU
!nvidia-smi

Tue Nov 29 12:11:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W /  70W |   1198MiB / 15109MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
### training time - takes around 15 mins

trainer.train()
### save the model for later use

trainer.save_model(Working_Dir + "/tuned_models/BART_models/" + experiment_name)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 750
  Number of trainable parameters = 139420416
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.027785,71.146800,56.972200,62.217800,62.135100,55.820000
2,1.399300,0.010027,72.904800,59.642300,64.122900,64.037300,54.660000
3,1.399300,0.009856,72.439000,58.591300,63.251800,63.202300,55.460000


***** Running Evaluation *****
  Num examples = 50
  Batch size = 1
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in output/checkpoint-500/tokenizer_config.json
Special tokens file saved in output/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/gdrive/Shareddrives/GOT/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/events12_community_event
Configuration saved in /content/gdrive/Shareddrives/GOT/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/events12_community_event/config.json
Model weights saved in /content/gdrive/Shareddrives/GOT/Building_Narratives_Game_of_Thrones

In [ ]:
### let's have a look at the predictions

preds, labels, metrics = trainer.predict(tokenize_data['test'], num_beams=5, min_length=50, max_length=128, no_repeat_ngram_size=2, early_stopping=True)

***** Running Prediction *****
  Num examples = 50
  Batch size = 1


In [ ]:
pred = []
lab = []
for gen, gold in zip(preds, labels):
  gen = tokenizer.decode(gen, skip_special_tokens=True)
  gen = str(gen)
  gen = gen
  pred.append(gen)
  #print(pred)
  #print(f'Generated text: {gen}')
  gold = tokenizer.decode(gold, skip_special_tokens=True)
  gold = str(gold)
  gold = [gold]
  lab.append(gold)
  #results = bleu.compute(predictions=gen, references=gold)
  #print(results)
  #print(f'Reference text: {gold}')
print(len(pred))  
print(len(lab))

50
50


In [ ]:

bleu = evaluate.load("bleu")
result_bleu= bleu.compute(predictions=pred, references=lab)
print(result_bleu)

google_bleu = evaluate.load("google_bleu")
result_google_bleu = google_bleu.compute(predictions=pred, references=lab)
print(result_google_bleu)

{'bleu': 0.47937384965276164, 'precisions': [0.7883035282745288, 0.6295195641406637, 0.5129507364144236, 0.4486711829077645], 'brevity_penalty': 0.8246083060912132, 'length_ratio': 0.8383306320907618, 'translation_length': 2069, 'reference_length': 2468}
{'google_bleu': 0.4712677725118483}


In [ ]:
experiment_name

'events12_community_event'

In [ ]:
#os.mkdir(Working_Dir + '/generated_text/by_BART/' + experiment_name )
outfile_path = Working_Dir + '/generated_text/by_BART/' + experiment_name + '/generated_text_with_refs_bleu.txt'

outfile = open(outfile_path, "a", encoding='utf-8')

scores = metrics.items()
print(f'Results: {scores}')
print(f'Bleu Results: {result_bleu}')
print(f'Google Bleu Results: {result_google_bleu}')

outfile.write(f'Test Results: {scores}\n\n')
outfile.write(f'Bleu Results: {result_bleu}\n\n')
outfile.write(f'Google Bleu Results: {result_google_bleu}\n\n')

for gen, gold in zip(preds, labels):
  gen = tokenizer.decode(gen, skip_special_tokens=True)
  print(f'Generated text: {gen}')
  outfile.write(f'Generated text: {gen}\n')

  gold = tokenizer.decode(gold, skip_special_tokens=True)
  print(f'Reference text: {gold}')
  outfile.write(f'Reference text: {gold}\n\n')
  
outfile.close()

Results: dict_items([('test_loss', 0.009503817185759544), ('test_rouge1', 73.6822), ('test_rouge2', 61.006), ('test_rougeL', 63.9183), ('test_rougeLsum', 64.0274), ('test_gen_len', 58.4), ('test_runtime', 32.2638), ('test_samples_per_second', 1.55), ('test_steps_per_second', 1.55)])
Bleu Results: {'bleu': 0.47937384965276164, 'precisions': [0.7883035282745288, 0.6295195641406637, 0.5129507364144236, 0.4486711829077645], 'brevity_penalty': 0.8246083060912132, 'length_ratio': 0.8383306320907618, 'translation_length': 2069, 'reference_length': 2468}
Google Bleu Results: {'google_bleu': 0.4712677725118483}
Generated text:  In Great Sept of Baelor Bran Stark was a  King from the House Tyrell and worked as a soldier. It was the Autumnal Equinox in Seagard when Daenerys Targaryen threatened the  jewel of Jon Stark. 
Reference text: Great Sept of Baelor was the place where Bran Stark, the King of House Tyrell used to live. Bran Stark was also known around Great Sept of Baelor as a great soldie

In [ ]:
for i in metrics.items():
  print(i)

('test_loss', 0.009503817185759544)
('test_rouge1', 73.6822)
('test_rouge2', 61.006)
('test_rougeL', 63.9183)
('test_rougeLsum', 64.0274)
('test_gen_len', 58.4)
('test_runtime', 34.5931)
('test_samples_per_second', 1.445)
('test_steps_per_second', 1.445)
